## <b><font color='darkblue'>[Intro to Building and Deploying an Agent with Agent Engine in Vertex AI](https://www.skills.google/course_templates/1378/documents/593367)</font></b>
([source](https://colab.research.google.com/drive/1D455tSBK21uGblumMCkzhaVnPap4vhyK?resourcekey=0-B6tDzoRMDLX7etXxidFw8Q&usp=drive_link#scrollTo=VEqbX8OhE8y9)) <font size='3ptx'><b>Agent Engine is a managed service that helps you to build and deploy an agent framework.</b> It gives you the flexibility to choose how much reasoning you want to delegate to the LLM and how much you want to handle with customized code.</font>

Agent Engine integrates closely with the Python SDK for the Gemini model in Vertex AI, and it can manage prompts, agents, and examples in a modular way. Agent Engine is compatible with LangChain, LlamaIndex, or other Python frameworks.

### <b><font color='darkgreen'>Objectives</font></b>
<font size='3ptx'><b>In this tutorial, you will learn how to build and deploy an agent (model, tools, and reasoning) using the Vertex AI SDK for Python.</b></font>

You'll build and deploy an agent that uses the Gemini model, Python functions as tools, and LangChain for orchestration.

You will complete the following tasks:
* Install the Vertex AI SDK for Python
* Use the Vertex AI SDK to build components of a simple agent
* Test your agent locally before deploying
* Deploy and test your agent on Vertex AI
* Customize each layer of your agent (model, tools, orchestration)

### <b><font color='darkgreen'>Getting Started</font></b>

#### <b><font size='3ptx'>Install Vertex AI SDK for Python</font></b>
Install the latest version of the Vertex AI SDK for Python as well as extra dependencies related to Agent Engine and LangChain:
```shell
%pip install --upgrade --quiet \
    "google-cloud-aiplatform[agent_engines,langchain]" \
    cloudpickle==3.0.0 \
    "pydantic>=2.10" \
    requests
```

In [4]:
!pip freeze | grep -P "(google-cloud|cloudpickle|pydantic|requests)"

cloudpickle==3.0.0
google-cloud-aiplatform==1.133.0
google-cloud-appengine-logging==1.6.2
google-cloud-audit-log==0.3.2
google-cloud-bigquery==3.36.0
google-cloud-bigquery-storage==2.35.0
google-cloud-bigtable==2.32.0
google-cloud-core==2.4.3
google-cloud-discoveryengine==0.13.12
google-cloud-logging==3.12.1
google-cloud-monitoring==2.28.0
google-cloud-resource-manager==1.14.2
google-cloud-secret-manager==2.24.0
google-cloud-spanner==3.57.0
google-cloud-speech==2.33.0
google-cloud-storage==2.19.0
google-cloud-trace==1.16.2
pydantic==2.12.5
pydantic-settings==2.10.1
pydantic_core==2.41.5
requests==2.32.5
requests-toolbelt==1.0.0


### <b><font color='darkgreen'>Restart current runtime</font></b>
To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [6]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
# from .autonotebook import tqdm as notebook_tqdm
# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### <b><font color='darkgreen'>Set Google Cloud project information and initialize Vertex AI SDK</font></b>
<b><font size='3ptx'>To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://www.google.com/url?q=https%3A%2F%2Fconsole.cloud.google.com%2Fflows%2Fenableapi%3Fapiid%3Daiplatform.googleapis.com).</font></b>

Learn more about [setting up a project and a development environment](https://docs.cloud.google.com/vertex-ai/docs/start/cloud-environment?hl=zh-tw).

In [7]:
from dotenv import load_dotenv
import os
from pathlib import Path
import vertexai

# Load environment variables from .env file
load_dotenv(Path('.') / '.env')

PROJECT_ID = os.getenv('PROJECT_ID')
LOCATION = os.getenv('LOCATION')
STAGING_BUCKET = os.getenv('STAGING_BUCKET')
API_KEY = os.getenv('API_KEY')

If you are working on local, you have to refer to [**this doc**](https://docs.cloud.google.com/docs/authentication/gcloud#local) and do authorization with commands as below:
```shell
$ gcloud auth login
```

In [8]:
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET)

# --- Initialization with API Key ---
#try:
#    # Initialize the client using the API key
#    client = vertexai.Client(api_key=API_KEY)
#    print("Vertex AI client initialized successfully with API key.")
#except Exception as e:
#    print(f"An error occurred: {e}")

### <b><font color='darkgreen'>Example: Build and deploy an agent</font></b>

#### <b><font size='3ptx'>Import libraries</font></b>

In [9]:
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import LangchainAgent

#### <font size='3ptx'><b>Define model</b></font>
As you construct your agent from the bottom up, the first component deals with which generative model you want to use in your agent.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-1.png" alt="Components of an agent in Agent Engine on Vertex AI" />

Here you'll use the Gemini 2.0 model:

In [10]:
from vertexai import generative_models

MODEL_NAME = "gemini-2.0-flash"
# model = client.generative_models.GenerativeModel(MODEL_NAME)

#### <B><font size='3ptx'>Define Python functions (tools)</font></b>
The second component of your agent includes tools and functions, which enable the generative model to interact with external systems, databases, document stores, and other APIs so that the model can get the most up-to-date information or take action with those systems.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-2.png" alt="Components of an agent in Agent Engine on Vertex AI" />

In this example, you'll define a function called `get_exchange_rate` that uses the `requests` library to retrieve real-time currency exchange information from an API:

In [11]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

Test the function with sample inputs to ensure that it's working as expected:

In [12]:
get_exchange_rate(currency_from="USD", currency_to="CNY")

{'amount': 1.0, 'base': 'USD', 'date': '2026-01-15', 'rates': {'CNY': 6.9673}}

#### <b><font size='3ptx'>Define model</font></b>
As you construct your agent from the bottom up, the first component deals with which generative model you want to use in your agent.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-1.png" alt="Components of an agent in Agent Engine on Vertex AI" />

Here you'll use the Gemini 2.0 model:

In [14]:
MODEL = "gemini-2.0-flash"

#### <b>Define agent</b>
The third component of your agent involves adding a reasoning layer, which helps your agent use the tools that you provided to help the end user achieve a higher-level goal.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-3.png" alt="Components of an agent in Agent Engine on Vertex AI" />

If you were to use Gemini and Function Calling on their own without a reasoning layer, you would need to handle the process of calling functions and APIs in your application code, and you would need to implement retries and additional logic to ensure that your function calling code is resilient to failures and malformed requests.

Here, you'll use the LangChain agent template provided in the Vertex AI SDK for Agent Engine, which brings together the model, tools, and reasoning that you've built up so far:

In [15]:
agent = agent_engines.LangchainAgent(
    model=MODEL,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

Now we can test the model and agent behavior to ensure that it's working as expected before we deploy it:

#### <b><font size='3ptx'>Test your agent locally</font></b>
With all of the core components of your agent in place, you can send a prompt to your agent using `.query` to test that it's working as expected, including the intermediate steps that the agent performed between the input prompt and the generated summary output.

In the default mode, the agent processes your input and returns the **entire agent output in a single response when complete**:

In [16]:
agent.query(input="What's the exchange rate from US dollars to Indian currency latest ?")

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'input': "What's the exchange rate from US dollars to Indian currency latest ?",
 'output': 'The current exchange rate from USD to INR is 1 USD to 90.3 INR.',
 'intermediate_steps': [[{'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'],
    'kwargs': {'tool': 'get_exchange_rate',
     'tool_input': {'currency_from': 'USD', 'currency_to': 'INR'},
     'log': "\nInvoking: `get_exchange_rate` with `{'currency_from': 'USD', 'currency_to': 'INR'}`\n\n\n",
     'type': 'AgentActionMessageLog',
     'message_log': [{'lc': 1,
       'type': 'constructor',
       'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'],
       'kwargs': {'content': '',
        'additional_kwargs': {'function_call': {'name': 'get_exchange_rate',
          'arguments': '{"currency_from": "USD", "currency_to": "INR"}'}},
        'response_metadata': {'safety_ratings': [],
         'usage_metadata': {},
         'finish_reason': 'STOP',
         'model_name': 'gemi

In addition to the default query mode, the `.stream_query` method allows you to **see the agent's intermediate steps and final output from the chain**.

Instead of waiting for the agent to complete all sub-tasks, the agent sends back the response in **chunks as it's being generated**:

In [17]:
message_types = {"actions": "Action", "messages": "Message", "output": "Output"}
for chunk in agent.stream_query(
    input="What's the exchange rate from US dollars to Indian currency latest ?"
):
    for key, label in message_types.items():
        if key in chunk:
            print("\n------\n")
            print(f"{label}:")
            print()
            print(chunk[key])


------

Action:

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'], 'kwargs': {'tool': 'get_exchange_rate', 'tool_input': {'currency_to': 'INR'}, 'log': "\nInvoking: `get_exchange_rate` with `{'currency_to': 'INR'}`\n\n\n", 'type': 'AgentActionMessageLog', 'message_log': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_exchange_rate', 'arguments': '{"currency_to": "INR"}'}}, 'response_metadata': {'safety_ratings': [], 'usage_metadata': {}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash'}, 'type': 'AIMessageChunk', 'id': 'run--019bc443-865a-7c31-b070-f4ac413c0e65', 'tool_calls': [{'name': 'get_exchange_rate', 'args': {'currency_to': 'INR'}, 'id': '090778e6-baf4-4f87-9673-6b291459bc85', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 48, 'output_tokens': 9, 'total_tokens': 57, 'input_token_details': {'c

This allows you to observe the agent's actions in real-time (such as function calls, and intermediate steps), which is helpful for debugging purposes or for providing real-time updates to the end user.

#### <b><font size='3ptx'>Deploy your agent on Vertex AI</font></b>
Now that you've specified a model, tools, and reasoning for your agent and tested it out, you're ready to deploy your agent as a remote service in Vertex AI!

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-4.png" alt="Components of an agent in Agent Engine on Vertex AI" />

You can re-define the agent to avoid any stateful information in the agent due to our testing in the previous cell:

In [18]:
agent = LangchainAgent(
    model=MODEL,
    tools=[get_exchange_rate],
)

Now you're ready to deploy your agent to Agent Engine in Vertex AI by calling `agent_engines.create()` along with:
1. The instance of your agent class
2. The Python packages and versions that your agent requires at runtime, similar to how you would define packages and versions in a `requirements.txt` file.

In [19]:
remote_agent = agent_engines.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
        "requests",
    ],
    display_name="Currency Exchange Agent",
)

Identified the following requirements: {'google-cloud-aiplatform': '1.133.0', 'pydantic': '2.12.5', 'cloudpickle': '3.0.0'}
The final list of requirements: ['google-cloud-aiplatform[agent_engines,langchain]', 'cloudpickle==3.0.0', 'pydantic>=2.10', 'requests']
Using bucket test_agent_staging
Wrote to gs://test_agent_staging/agent_engine/agent_engine.pkl
Writing to gs://test_agent_staging/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://test_agent_staging/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/1056070812087/locations/us-central1/reasoningEngines/1353381166048083968/operations/7820153152635666432
View progress and logs at https://console.cloud.google.com/logs/query?project=pwt-ai-center
AgentEngine created. Resource name: projects/1056070812087/locations/us-central1/reasoningEngines/1353381166048083968
To use this AgentEngine in another session:
agent_engine = vertexai.agent_engines.get('pro

Now you can send a prompt to your remote agent using `.query` to test that it's working as expected:

In [20]:
remote_agent.query(
    input="What's the exchange rate from US dollars to Indian currency latest ?"
)

{'input': "What's the exchange rate from US dollars to Indian currency latest ?",
 'output': 'The exchange rate from USD to INR is 90.3.'}

Or you can stream the results back from the remote agent interactively using `.stream_query`...

#### <b><font size='3ptx'>Querying your deployed agent</font></b>
You've now deployed your agent and can [interact with it in multiple ways](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/overview), both within this notebook and from other applications or environments. The primary methods for accessing your deployed agent are via the Python client library or through REST API calls. Here's an overview of both methods:

In [21]:
# List all agent engines
all_agent_engines = agent_engines.list()
print("All Agent Engines:")
for agent in all_agent_engines:
    print(f"- {agent.display_name} : {agent.resource_name}")

All Agent Engines:
- Currency Exchange Agent : projects/1056070812087/locations/us-central1/reasoningEngines/1353381166048083968


Use the resource name to load the agent in your other notebook or Python script, then query the remote agent as usual:

In [22]:
RESOURCE_ID="1353381166048083968"
remote_agent = agent_engines.get(RESOURCE_ID)

In [23]:
remote_agent.query(
    input="What's the exchange rate from US dollars to Indian currency latest ?"
)

{'input': "What's the exchange rate from US dollars to Indian currency latest ?",
 'output': 'I am sorry, I cannot fulfill this request. The exchange rate from USD to INR is 90.3.'}

#### <b><font size='3ptx'>Cleaning up</font></b>
<b><font size='3ptx'>After you've finished, it's a good practice to clean up your cloud resources.</font></b> You can delete the deployed Agent Engine instance to avoid any unexpected charges on your Google Cloud account.

In [24]:
remote_agent.delete()

Delete Agent Engine backing LRO: projects/1056070812087/locations/us-central1/operations/2374456778214604800
Agent Engine deleted. Resource name: projects/pwt-ai-center/locations/us-central1/reasoningEngines/1353381166048083968


## <b><font color='darkblue'>Supplement</font></b>
* [Authenticate for using the gcloud CLI (Local environment)](https://docs.cloud.google.com/docs/authentication/gcloud#local)